# Generate second order thanks dataset for final analysis (NO LONGER IN USE)
    J. Nathan Matias, March 2021

In [41]:
import csv, os, sys, datetime
from collections import Counter, defaultdict
import pandas as pd

In [5]:
anonymous_ids = {}

if 'TRESORDIR' in os.environ:
    data_path = os.environ['TRESORDIR']
else:
    data_path = os.path.expanduser("~") + '/Tresors'


## Load Anonymous IDs

In [6]:
user_name_anon_ids = {}
with open(data_path + "/CivilServant/projects/wikipedia-integration/gratitude-study/datasets/misc/thankee_public_anonymous_ids_20200212.csv") as f:
   for row in csv.DictReader(f):
       user_name_anon_ids[row['user_name']] = row['public_anonymous_id']

## Load Second Order Thanks

In [7]:

second_order_thanks = []
thank_filename = 'gratitude-second-gen-thanks-analysis-with-reciprocal.csv'
filepath = "CivilServant/projects/wikipedia-integration/gratitude-study/Data Drills/thankee/post_experiment_analysis"

with open(os.path.join(data_path, filepath, thank_filename)) as f:
    for row in csv.DictReader(f):
        new_row = {}
        for col in ['second.gen.thank.ts', 'lang','behavior.start.dt','second.gen.recipient.is.first.gen.sender']:
            new_row[col] = row[col]
        if 'user.name' in user_name_anon_ids.keys():
            new_row['public_anonymous_id'] = user_name_anon_ids[row['user.name']]
        else:
            new_row['public_anonymous_id'] = None
        second_order_thanks.append(row)

# Load Username to Experiment Condition Dataset

In [8]:
uname_condition_fname = 'gratitude-thankee-username-arms.csv'

uname_condition = {}

with open(os.path.join(data_path, filepath, uname_condition_fname)) as f:
    for row in csv.DictReader(f):
        uname_condition[row['username']] = row['arm']

## How often do we have a match on the user names?


In [9]:
print("{0} unique usernames making {1} thanks".format(len(set([x['user.name'] for x in second_order_thanks])), len(second_order_thanks)))
print("{0} usernames in the index".format(len(uname_condition)))

488 unique usernames making 2552 thanks
15429 usernames in the index


In [11]:
thanks_by_participant = defaultdict(list)

for thank in second_order_thanks:
    thank['condition'] = uname_condition[thank['user.name']]
    thanks_by_participant[thank['user.name']].append(thank)

print("{0} total participants with thanks".format(len(thanks_by_participant)))

488 total participants with thanks


In [12]:
## Take in a list of thanks sent by an account
## and create a single row for the analysis
def generate_analysis_row(thanks_list):
    total_thanks = 0
    reciprocal_thanks = 0
    nonreciprocal_thanks = 0
    lang = None
    for thank in thanks_list:
        total_thanks += 1
        lang = thank['lang']
        if(thank['second.gen.recipient.is.first.gen.sender']=="True"):
            reciprocal_thanks += 1
        else:
            nonreciprocal_thanks += 1
    account_row = {
        "condition": thank['condition'],
        "identifiable.thanks.sent": total_thanks,
        "reciprocal.thanks.sent": reciprocal_thanks,
        "nonreciprocal.thanks.sent": nonreciprocal_thanks,
        "lang": lang
    }
    return account_row

In [13]:
participant_thanks = []

for thanks_list in thanks_by_participant.values():
    participant_thanks.append(generate_analysis_row(thanks_list))    

In [14]:
## check that the code works
Counter([x['reciprocal.thanks.sent'] for x in participant_thanks])

Counter({0: 482, 1: 6})

# Write to File

In [51]:
thank_analysis_filename = datetime.datetime.strftime(datetime.datetime.utcnow(), format="%Y-%M-%d-secondary-thanks.csv")#

pd.DataFrame(participant_thanks).to_csv(os.path.join(data_path, filepath, thank_analysis_filename), index=False)